In [182]:
import os
import random
import re

from googleapiclient.discovery import build
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, auc, brier_score_loss, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Summary

The task is to classify which industry a company belongs to given the name of a company. The dataset contains approximately 300,000 companies, spanning 99 industries. The dataset only contains two columns which is the name and the true proper classification of the industry it belongs to. This is unique because a) rather than having a wide dataset, the dataset is small in terms of features and b) conventional methods of data analysis would not apply since I do not have enough features and would have to engineer them myself.

The steps I took to address this issue are:
    * Generate the features
    * Transform them to be meaningful
    * Eliminate redundant or noisy features
    * Train the data and assess the model
    
To begin, here is a quick look at the data set that was given:

In [7]:
data = pd.read_csv('/Users/cipherpol/Downloads/private_us_companies.csv',
                   encoding='latin1', usecols=['Company', 'Industry'])
data.head()

,Company,Industry
0,A - E Employees Credit Union,Diversified Financial Services
1,"A & A Contract Services, Inc.",Professional Services
2,"A & A Express, Inc.",Road and Rail
3,"A & A Fertilizer, Ltd.",Chemicals
4,A & A Food Service. Inc.,Distributors


# Feature Generation

Since the dataset lacks features to describe the classification, I would have to generate some. My first thought is to parse the name and hopefully find some meaningful patterns there, but a quick glance would show this would not work since  there are similar names such as `A & A Express, Inc` and `A & A Fertilizer, Ltd` which have `A & A` but are classified differently. For instance, I have no theoretical reason to believe that names that start with A would be more likely to be classified as Chemicals for instance.

Usually, the first thing most people do when faced with unfamiliar information is to Google that concept. I was able to find a website for most companies here and decided to go down this approach in order to generate features. I used the Google Custom Search API. The idea was to create a custom search using the [API](https://developers.google.com/custom-search/docs/tutorial/creatingcse) and search the company name. From there I would pick the most relevant search according to their PageRank algorithm and grabed the `htmlSnippet` and `htmlTitle`. The reason for this approach is that the `htmlSnippet` would describe the body of text for the company while the `htmlTitle` would be a quick summary of the company. Both of these are highly relevant.

I saved my api and custom search engine id (cse_id) and retrieved them from my environmental variables for security purposes, then I created a quick function to parse it.

    api_key = os.environ.get('google_api_key')
    cse_id = os.environ.get('cse_id')

    # custom search using the Google Search API
    def google_search(search_term, api_key, cse_id, **kwargs):
        service = build("customsearch", "v1", developerKey=api_key)
        res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
        result = res['items']
        return result[0]['htmlSnippet'] + " " + result[0]['htmlTitle']


An example is below

In [13]:
google_search('A & A Food Service. Inc.', my_api_key, my_cse_id)

'<b>A &amp; A Food Service Company</b> is a full service purveyor to the food industry. Our <br>\nclientele includes restaurants, convention centers, hospitals, high-end hotels,&nbsp;... <b>A &amp; A FOODSERVICE</b> - Welcome'

From there, I simply just pass it through some regex filters to clean up any HTML tags along with any unicode characters present and saved it as a dictionary for future processing.

    def clean_text(text):
        remove_tags = re.compile('(\<\/?\w+\>)')
        tags_gone= remove_tags.sub(' ', text)
        clean_up = re.compile('(&#39;_)|(\W+)')
        return clean_up.sub(' ', tags_gone).strip()
        
(**Disclaimer: The Google CSE API only allows 100 API requests a day and charges $5 for every** **1,000 queries up to 10,000 a day. Given these financial constraints, I only used a sample  of 100 companies to create this report**)

# Feature Extraction

Now that I have the text parsed and cleaned, I have to extract the features and convert it in a way that can be useful. My approach is as followed:
   * Count the occurrence of all the words
   * Create an array that represents the presence of a word for each company
   * Transform the array to represent the importance of the word
  
To accomplish this, I used the `CountVectorizer` which takes in a corpus of text and then with each company's description, you can get an array that contain the counts of each word. For instance, if the corpus is this (taken from the sklearn [documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#common-vectorizer-usage))

    corpus = [
        'This is the first document.',
        'This is the second second document.',
        'And the third one.',
        'Is this the first document?',
    ]
    X = vectorizer.fit_transform(corpus)
    
will be converted to

    array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]]...)
       
where the position of the numerical values correspond to each
 
     vectorizer.get_feature_names()
     (['and', 'document', 'first', 'is', 'one',
       'second', 'the', 'third', 'this']
      )
 
So in the first array, we see a `0` in the first element because there is no `and` but there is `document`, `first`, and `is` so the second, third, and fourth element in the array is represented with a `1`. From here, I simply just pass in the values of my dictionary

Now that we have vectorize the word counts, I wanted to re-weigh them. I used the [Tf–idf term](http://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting) method. The idea behind this technique is that words that are common such as `a` or `the` will have a lesser weights than words that are rare such as `Agriculture` or `Banking`. My theory is that words that are rarer will also be correlated with the group they belong to and I wanted to capture that relationship.

Below I ran a quick example of my theory and we can see the shape of the vector names and the word vector match up

In [ ]:
count_vectorizer = CountVectorizer()
inv_transformer = TfidfTransformer(smooth_idf=False)
vec = count_vectorizer.fit_transform(self.words.values()).toarray()
vector = inv_transformer.fit_transform(vec).toarray()
vector_names = count_vectorizer.get_feature_names()

In [67]:
vector.shape # a 50 x 696 matrix

(50, 696)

In [70]:
len(vector_names)

696

In [192]:
# a random selection of words
np.random.choice(np.array(self.vector_names), 20)

array(['electronic', 'cycling', 'around', 'jones', 'should', '2005', 'pty',
       'union', 'bamboo', '944', 'southwest', 'contact', 'kansas',
       'apartment', 'collins', 'huneeus', 'cases', 'york', 'holiday',
       'networks'], 
      dtype='<U14')

# Feature Selection

From the `CountVectorizer` we have a wide variety of words that describes these companies. This presents me with a problem that was the opposite at the beginning: we now have too many features and our data set is sparse. This means that the data set is populated with many 0 which makes sense because each word is drawn from a large corpus. With a sample of 50 companies, we have approximately 700 features represented.

Now my task is to reduce the values. To do this, I used two techniques: an ElasticNet regularizator and Principle Component Analysis.

The reason why I chose an [ElasticNet](https://web.stanford.edu/~hastie/Papers/B67.2%20(2005/\%\20301-320%20Zou%20&%20Hastie.pdf) is because it's a compromise between L1 and L2 regularization techiques. These regularizations places a penalty on wide feature sets and enforces parsimony of features. L1 regularization will find a group of variables that are correlated and randomly select one, while L2 will simply shrink the coefficients but never make them zero, thus preserving the structure of the current features. Elastic Net compromises between the two. The reason I chose this techinque is because I believe there is a grouping effect of variables. For instance, the work "Banking, Investment, and Credit" might indicate that it is a financial service industry.

The ElasticNet regularization can be called using the [SelectFromModel](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel) wrapper, which allows feature selection by models that have feature importance rankings

            select = SelectFromModel(ElasticNet())


Below I just implemented the code. First I ran a quick `LabelEncoder` that would assign a unique value for every industry (since the outcome cannot be a string) and then I scaled each column vector to a mean of 0 and a standard deviation of 1. We see that the 696 columns down to 157 columns

In [127]:
labelizer = LabelEncoder()
labelizer.fit(list(set(self.df.Industry)))
X, self.y = self.df.ix[:, 1:-1], labelizer.transform(self.df.Industry)
scalar = StandardScaler()
scaled = scalar.fit_transform(X)
scaled.shape


(50, 696)

In [129]:
select = SelectFromModel(ElasticNet())
selected = select.fit_transform(scaled, self.y)
selected.shape

(50, 157)

The second part is to run [Principle Component Analysis](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA) which will reduce the features into components that captures the highest variance in the data. The idea is to reduce the features while retaining variance. This part is depending on the client need. If the client would prefer interpretable features, then I would omit using PCA. However, if the concern is parsimony and predictive power, I would select this technique. Furthermore, reducing the number of features would improve computational speed, which would be useful as we scale up our analysis.

In this scenario, I did not specify the number of components, but I allow it to iterate until the projected components represent at least 95% of the variance of the original data set.

    pca = PCA(n_components=.95, svd_solver='full')
    
Here we see that we further reduced the variables from 157 to 15 that captures 95% of the variance!

In [131]:
pca = PCA(n_components=.90, svd_solver='full')
decomposed = self.X = pca.fit_transform(selected)
decomposed.shape

(50, 15)

# Training the Model

After feature selection, I can turn to the modeling process. I chose to implement a fairly popular model: [The Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). The reason I chose this model is that
    * it is robust to outliers,
    * I am assuming there is nonlinearity within the data
    * it is less likely to overfit since it only sample certain features each iteration
    * it **scales** every well since the bootstrapping can occur on separate nodes

In addition to these benefits, I can also implement a hyperparameter tuning methods to determine the best combination of parameters to pass into the Random Forest algorithm that would yield the lowest cross-validated error rates. This can be done by using the [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), where I can specify the parameters and values into a dictionary to be searched through.

In [175]:
params = {
     'n_estimators': [2, 3, 4, 5, 6],
     'max_features': [1, 2, 3, 4],
     'criterion': ['gini', 'entropy']}
rf = GridSearchCV(
     estimator=RandomForestClassifier(),
     param_grid=params,
     n_jobs=3,
     verbose=True)


Here the tuning grid will search through `n_estimators`, `max_features`, and `criterion` parameter and their respective values, which will yield 40 different combinations.

After training the model, depending on the business case, we could use several metrics. We can train the model to maximize specificity or sensitivity. I opted out from running a metrics examination because the training (and the test set) are small due to the API limit and I do not know the use case of the model. Do we care more about labeling industries correctly or minimizing wrong classifications? How do we intend to use the model? What subject-matter expertise can I draw from to improve the model? These are all important questions I ask before I decide on the proper metric to use for model evaluation.

# Conclusion

Putting everything together, we can create two classes: one for generating and parsing the features and one for extracting, transforming, and training. The reason why I create these classes because it will allow for changing states in our training process and train models with just two lines of code. Also, it will be easier to scale up if we build out into a web application (Django, for example).


```
class Parser:

    def __init__(self, list_of_words, api_key, cse_id):
        self.words = list_of_words
        self.api = api_key
        self.cse = cse_id

    @staticmethod
    def google_search(search_term, api_key, cse_id, **kwargs):
        service = build("customsearch", "v1", developerKey=api_key)
        res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
        result = res['items']
        return result[0]['htmlSnippet'] + " " + result[0]['htmlTitle']

    @staticmethod
    def clean_text(text):
        remove_tags = re.compile('(\<\/?\w+\>)')
        tags_gone= remove_tags.sub(' ', text)
        clean_up = re.compile('(&#39;_)|(\W+)')
        return clean_up.sub(' ', tags_gone).strip()

    def run(self):
        search_dict = {}
        for word in self.words:
            result = self.google_search(word, self.api, self.cse)
            cleaned = self.clean_text(result)
            search_dict[word] = cleaned
        self.word_dict = search_dict


class Analyzer:

    def __init__(self, word_dict, data):
        self.words = word_dict
        self.raw_df = data

    def prepare_feature_vector(self):
        count_vectorizer = CountVectorizer()
        inv_transformer = TfidfTransformer(smooth_idf=False)
        vec = count_vectorizer.fit_transform(self.words.values()).toarray()
        self.vector = inv_transformer.fit_transform(vec).toarray()
        self.vector_names = count_vectorizer.get_feature_names()

    def prepare_df(self):
        feat_df = pd.DataFrame(self.vector, columns=self.vector_names)
        company_df = pd.DataFrame(list(self.words.keys()))
        full_df = pd.concat([company_df, feat_df], ignore_index=True, axis=1)
        full_df.columns = ['Company'] + self.vector_names
        self.df = pd.merge(full_df, self.raw_df)

    def preprocess_data(self):
        labelizer = LabelEncoder()
        labelizer.fit(list(set(self.df.Industry)))
        X, self.y = self.df.ix[:, 1:-1], labelizer.transform(self.df.Industry)
        scalar = StandardScaler()
        scaled = scalar.fit_transform(X)
        select = SelectFromModel(ElasticNet())
        selected = select.fit_transform(X, self.y)
        pca = PCA(n_components=.90, svd_solver='full')
        self.X = pca.fit_transform(selected)

    def train_data(self, **kwargs):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=0.2, random_state=5)
        params = {
            'n_estimators': [2, 3, 4, 5, 6],
            'max_features': [1, 2, 3, 4],
            'criterion': ['gini', 'entropy']
        }
        if kwargs:
            for key, value in kwargs.items():
                params[key] = value
        self.rf = GridSearchCV(
            estimator=RandomForestClassifier(),
            param_grid=params,
            n_jobs=3,
            verbose=True)
        self.rf.fit(self.x_train, self.y_train)
        self.class_predict = self.rf.predict(self.x_test)
        self.probs_predict = self.rf.predict_proba(self.x_test)[:, 1]
        
    def get_metrics(self):
        self.report = classification_report(self.y_test, self.class_predict, 
        self.vector_names)
        
    def run(self):
        self.prepare_feature_vector()
        self.prepare_df()
        self.preprocess_data()
        self.train_data()

```

There are limitations that I wished to address. This project only assumes that the Google Search terms are correlated to the industry classification, which I believe is a reasonable assumption. And without being able to fully grab all the necessary data possible, I was not able to explore the data a bit more and build out a more robust model.

Other approaches I could have employed if I had more time:

  * Exploring the use of bi-grams and  tri-grams and seeing if they improve model performance
  * Appending census data once we are able to parse out the location and address of the business
  * Implementing a [Word2Vec](https://arxiv.org/pdf/1301.3781v3.pdf) algorithm  to understand the relationship between certain words
  * Using a [TruncatedSVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD) to decompose a spare matrix (I did not implement because I do not understand it fully)
  * Using the [XGBOOST](https://github.com/dmlc/xgboost) Model for classification, since it is one of the best supervised learning algorithms used (I did not implement since I wanted to stay inside the Sklearn framework)
  * Implementing a recurrent neural network to help with text classification since the location and context of the world most likely will have an impact on industry classification